Get access to the workspace

In [ ]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    credential.get_token("https://management.azure.com/.default")
except Exception as e:
    credential = InteractiveBrowserCredential()

In [ ]:
from azure.ai.ml import MLClient

ml_client = MLClient.from_config(credential=credential)

Upload the data as an MLTable

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

local_data_path = "data/diabetes.csv"

my_data = Data(
    path=local_data_path,
    type=AssetTypes.URI_FILE,
    description="Diabetes data",
    name="diabetes_data_uploaded"
)

ml_client.data.create_or_update(my_data)

In [ ]:
uploaded_data_path = ml_client.data.get(name="diabetes_data_uploaded", version=1).path

In [ ]:
import mltable

data_paths = [{"file": uploaded_data_path}]
table = mltable.from_delimited_files(data_paths, header='all_files_same_headers', delimiter=',', support_multi_line=False,
                                     empty_as_string=False, encoding='utf8', include_path_column=False, infer_column_types=True)
table.show(5)

In [ ]:
table.save("data/diabetes_mltable")

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

my_data = Data(
    path="data/diabetes_mltable/",
    type=AssetTypes.MLTABLE,
    description="Diabetes data",
    name="diabetes_data_mltable",
    version="2",
)

ml_client.data.create_or_update(my_data)

In [ ]:
mltable_data_path = ml_client.data.get(name="diabetes_data_mltable", version=2).path

Train model using automl

In [ ]:
from azure.ai.ml import automl, Input

training_data_input = Input(type=AssetTypes.MLTABLE, path=mltable_data_path)

classification_job = automl.classification(
    compute="aml-cluster",
    experiment_name="diabetes-automl-classification",
    training_data=training_data_input,
    target_column_name="Diabetic",
    primary_metric="AUC_weighted",
    n_cross_validations=5,
    enable_model_explainability=True
)

classification_job.set_limits(
    timeout_minutes=120,
    trial_timeout_minutes=15,
    max_trials=5,
    enable_early_termination=True
)

In [ ]:
returned_job = ml_client.jobs.create_or_update(classification_job)

returned_job.services["Studio"].endpoint